In [ ]:
import pandas as pd
import numpy as np
import os
import time
import json
import sys
import pickle
import argparse
import arviz as az
from pathlib import Path
from tqdm import tqdm
from pathlib import Path
from scipy.stats import norm, entropy
from utils import *

In [ ]:
transform_label = transforms_labels()
plt.rcParams["figure.figsize"] = [20,10]
fig, axes = plt.subplots(3, 3)

evaluating_model='DirichletSymmetric'
output_dir='output'
n_repeat=100

with open(f"target_densities/param_map_{evaluating_model}.pkl", "rb") as f:
    param_map = pickle.load(f)

for ax, parameters in zip(axes.flatten(), dirichletsymmetricparams()):
    for transform in tqdm(list_transforms()):
        try:
            complete_output_dir=f'{output_dir}/{transform}/{evaluating_model}'
            pkey=param_map[tuple(list(parameters.values())[0])]
            fstr = lambda *, x, fileformat: f"{complete_output_dir}/{x}_{pkey}_{n_repeat}.{fileformat}"
            leapfrog=pd.read_csv(fstr(x="leapfrog", fileformat="csv")).drop(columns=["chain"])
            x = np.mean(np.cumsum(leapfrog, axis=1))
            y = np.loadtxt(fstr(x="rmse", fileformat="txt"))
            ax.set_title(f"alpha={parameters['alpha'][0]}, N = {parameters['N']}")
            ax.plot(x,y, label=transform_label[str(transform)])
            
        except FileNotFoundError:
            print(f' no sampler data for parametrization {parameters} and transform {transform_label[str(transform)]}')

handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(),loc='upper center', bbox_to_anchor=(0.5, 0.97),
ncol=4, fancybox=True, shadow=True)


ax.axes.yaxis.set_ticklabels([])
fig.supxlabel('Cumulative Leapfrog Steps')
fig.supylabel('Root Mean Squared Error')
plt.savefig(f'figures/RMSE_{evaluating_model}.png', dpi=300)

In [ ]:
import matplotlib.pyplot as plt 
transform_label = transforms_labels()
plt.rcParams["figure.figsize"] = [20,10]
fig, axes = plt.subplots(3, 3)

evaluating_model='DirichletSymmetric'
output_dir='output'
n_repeat=100

with open(f"target_densities/param_map_{evaluating_model}.pkl", "rb") as f:
    param_map = pickle.load(f)

for ax, parameters in zip(axes.flatten(), dirichletsymmetricparams()):
    for transform in tqdm(list_transforms()):
        try:
            complete_output_dir=f'{output_dir}/{transform}/{evaluating_model}'
            pkey=param_map[tuple(list(parameters.values())[0])]
            fstr = lambda *, x, fileformat: f"{complete_output_dir}/{x}_{pkey}_{n_repeat}.{fileformat}"
            leapfrog=pd.read_csv(fstr(x="leapfrog", fileformat="csv")).drop(columns=["chain"])
            lp = leapfrog.rolling(4, min_periods=1, step=4).mean()
            ess = np.loadtxt(fstr(x="ess", fileformat="txt"))
            x = np.sort(ess/lp.sum(axis=1))
            y = np.arange(len(x))/float(len(x))
            ax.set_title(f"alpha={parameters['alpha'][0]}, N = {parameters['N']}")
            ax.plot(x,y, label=transform_label[str(transform)])
            
        except FileNotFoundError:
            print(f' no sampler data for parametrization {parameters} and transform {transform_label[str(transform)]}')

handles, labels = fig.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(),loc='upper center', bbox_to_anchor=(0.5, 0.97),
ncol=4, fancybox=True, shadow=True)


ax.axes.yaxis.set_ticklabels([])
fig.supxlabel('Cumulative Leapfrog Steps')
fig.supylabel('Root Mean Squared Error')
plt.savefig(f'figures/ess_{evaluating_model}.png', dpi=300)

In [ ]:

stan_filename=f'stan_models/{transform}_{evaluating_model}.stan'

start_time = time.time()

Path("stan_models").mkdir(parents=True, exist_ok=True)

with open(stan_filename, 'w') as f:
    f.write(f'#include ../target_densities/{evaluating_model}.stan{os.linesep}#include ../transforms/{transform}.stan{os.linesep}')
    f.close()